# Chpater 10 Maps, Hash Table,s and Skip Lists
## 10.1 Maps and Dictionaries
파이썬에서 가장 중요한 자료 구조를 고르라면 당연히 **dict** 클래스일 것이다. 파이썬 딕셔너리는 **딕셔너리**라고 알려진 추상화를 제공하는데, 이 딕셔너리에서는 유니크한 **key**가 관련된 **value**로 매핑된다. 딕셔너리가 나타내는 key와 value 사이의 관계로 인해 딕셔너리는 흔히 **연관 배열(associative arrays)** 혹은 **맵(maps)**이라고 불린다. 이 책에서는 파이썬의 `dict` 클래스를 지칭할때는 `dictionary`라 하고, 일반적인 자료 형의 개념을 다룰 때는 `map`이라 부를 것이다.

간단한 예제로, 아래의 그림은 국가의 이름과 통화의 단위를 연결한 맵의 예제이다.

<img width="600" alt="figure-10.1" src="https://user-images.githubusercontent.com/20944657/36952791-abd5636e-2057-11e8-9ee1-7df76f4f86b3.png">

key(국가 이름)는 유니크하지만 value(통화 단위)는 유니크할 필요가 없다. 예를 들어, 스페인과 그리스는 둘 다 통화 단위로 유로를 이용한다. 맵은 인덱싱을 위해 `currency['Greece']`와 같이 배열과 같은 문법을 이용한다. 이 때 주어진 key의 value를 새로운 value로 리매핑하려면 `currency['Greece'] = 'Drachma'`와 같은 문법을 이용할 수도 있다. 표준적인 배열과 다르게, 맵의 인덱스들은 연속적일 필요가 없고, 인덱스가 숫자여야 할 필요도 없다. 맵의 일반적인 활용은 다음과 같다.
- 대학의 정보 시스템(information system)은 학생 ID를 key로 이용해서 그 학생의 관련된 정보(이름, 주소, 학점)를 관리한다.
- DNS(domain-name system)는 `www.wiley.com`과 같은 호스트 네임을 IP(internet-protocol)에 매핑한다.
- 소셜 미디어 사이트는 (숫자가 아닌) 유저 이름을 key로 이용해서 유저의 관련된 정보에 효과적으로 매핑한다.
- 컴퓨터 그래픽 시스템은 `turquoise`와 같은 색 이름을 (64,224,208)과 같은 색의 RGB(red-green-blue) 숫자와 매핑한다.
- 파이썬은 딕셔너리를 이용해서 각각의 네임스페이스를 표현한다. 예를 들어 `pi`와 같은 식별자를 `3.14159`와 같은 객체로 연결한다.

우리는 이번 챕터와 다음 챕터에서 키를 이용해서 값을 찾는 탐색 과정이 매우 효율적으로 이루어질 수 있고 다양한 상황에서 활용될 수 있음을 보일 것이다.

### 10.1.1 The Map ADT
이번 섹션에서는 **맵 ADT(map ADT)**를 소개하고, 파이썬의 built-in `dict` 클래스와 일관성 있게끔 맵의 동작들을 정의할 것이다. 먼저 맵 $M$에서 가장 중요한 5가지 동작들을 살펴보자:
- **M[k]:** $M$에서 key `k`와 관련된 value `v`를 반환한다. 만약 없다면 `KeyError`가 발생한다. 파이썬에서는 이 메소드는 스페셜 메소드인 `__getitem__`으로 구현된다.
- **M[k] = v:** $M$에서 key `k`에 value `v`를 연결시킨다. 만약 이미 key `k`와 연결된 value가 있다면 새로운 value로 교환한다. 파이썬에서는 스페셜 메소드인 `__setitem__`으로 구현된다.
- **del M[k]:** 맵 $M$에서 key가 `k`인 항목을 제거한다. 만약 $M$에 그런 항목이 없다면 `KeyError`가 발생한다. 파이썬에서는 스페셜 메소드인 `__delitem__`으로 구현된다.
- **len(M):** 맵 $M$에 있는 항목들의 수를 반환한다. 파이썬에서는 스페셜 메소드인 `__len__`으로 구현된다.
- **iter(M):** 맵의 `keys` 시퀀스를 생성하는 맵의 iteration을 반환한다. 파이썬에서는 스페셜 메소드인 `__iter__`으로 구현된다.

위의 다섯가지 동작은 맵의 핵심적인 기능인 쿼리, 추가, 수정, key-value 페어의 제거, 이 페어들을 모두 보고하는 능력을 모두 설명하기에 중요하다. 추가적인 편의를 위해 맵 $M$은 다음의 동작들을 지원해야 한다:
- **k in M:** 맵이 key `k`인 항목을 포함하고 있다면 True를 반환한다. 파이썬에서는 스페셜 메소드인 `__contains__`으로 구현된다.
- **M.get(k, d=None):** 맵에 key `k`가 존재하면 `M[k]`를 반환한다. 만약 없다면 default value인 `d`를 반환한다. 이 메소드는 `KeyError`에 대한 걱정 없이도 `M[k]`를 쿼리할 수 있게 해준다.
- **M.setdefault(k, d):** 만약 맵 안에 key `k`가 존재한다면 `M[k]`를 반환한다. 만약 키 `k`가 존재하지 않는다면 `M[k]`를 `d`로 만들고 그 값을 반환한다.
- **M.pop(k, d=None):** 맵에서 key `k`와 연결된 항목을 제거하고 그 value `v`를 반환한다. 만약 key `k`가 맵 안에 없다면 기본 값 `d`를 반환한다 (만약 매개변수 `d`가 `None`이면 `KeyError`를 반환한다).
- **M.popitem():** 맵에서 임의의 key-value 페어를 제거하고 그 `(k,v)` 튜플을 반환한다. 맵이 비어있다면 `KeyError`가 발생한다.
- **M.clear():** 맵에서 모든 key-value 페어를 제거한다
- **M.keys():** $M$의 모든 key에 대한 set-like view를 반환한다.
- **M.values():** $M$의 모든 value에 대한 set-like view를 반환한다.
- **M.items():** $M$의 모든 항목의 (k,v) 튜플에 대한 set-like view를 반환한다.
- **M.update(M2):** 맵 $M2$의 모든 (k,v) 페어에 대해 `M[k] = v`로 할당한다.
- **M == M2:** 맵 $M$과 $M2$가 동일한 key-value 연결관계를 갖고 있을 때 True를 반환한다.
- **M != M2:** 맵 $M$과 $M2$가 동일하지 않은 key-value 연결관계를 갖고 있을 때 True를 반환한다.

### 10.1.2 Application: Counting Word Frequencies
맵을 이용하는 대표적인 사례로 문서에서 단어의 사용 빈도를 카운트하는 예제를 보자. 자세한 설명은 생략한다.

In [2]:
def wordcount(filename):
    freq = {}
    for piece in open(filename).read().lower().split():
        # only consider alphabetic characters within this piece
        word = ''.join(c for c in piece if c.isalpha())
        if word:
            freq[word] = 1 + freq.get(word, 0)
            
    max_word = ''
    max_count = 0
    for (w,c) in freq.items():
        if c > max_count:
            max_word = w
            max_count = c
    print('The most frequent word is', max_word)
    print('Its number of occurences is', max_count)

### 10.1.3 Python's MutableMapping Abstract Base Class
Section 2.4.3에서는 **추상 기초 클래스(abstract base class)**의 개념을 소개하고, 파이썬의 `collections` 모듈에서 그러한 클래스들의 역할을 설명했었다. 이러한 기초 클래스에서 추상 메소드로 선언된 메소드들은 구상 서브클래스에서 반드시 구현되어야 한다. 그러나 추상 기초 클래스는 이 추상 메소드들을 이용하는 다른 메소드들의 *구체적인* 구현을 제공하기도 한다(이는 **템플릿 메소드 디자인 패턴(template method design pattern**의 예제이다).

`collections` 모듈은 우리의 현재 논의와 관련된 두개의 추상 기초 클래스를 제공한다(`Mapping`과 `MutableMapping` 클래스). `Mapping` 클래스는 파이썬의 `dict` 클래스에 의해 지원되는 모든 nonmutating 메소드를 포함하고 있고, `MutableMapping` 클래스는 이를 mutating 메소드까지 포함하게끔 확장시킨다. 우리가 앞에서 맵 ADT라고 정의한 것은 파이썬의 `collections` 모듈 중 `MutableMapping` 추상 기초 클래스와 유사하다.

이러한 추상 기초 클래스의 중요성은 이 클래스가 사용자-정의 맵 클래스를 만드는 데 도움이 된다는 것에 있다. 구체적으로 말하면, `MutableMapping` 클래스는 처음 말한 5개의 `__getitem__`, `__setitem__`, `__delitem__`, `__len__`, `__iter__`를 제외하고는 모든 동작들에 대한 구체적인 구현을 제공한다. 다양한 자료 구조를 이용해서 맵의 추상화를 구현할 때 우리는 이 5개의 핵심 동작만 정의하면 나머지 동작들은 모두 `MutableMapping` 부모 클래스에서 상속받을 수 있다.

`MutableMapping` 클래스를 더 잘 이해하기 위해서 어떻게 구체적인 동작들이 5개의 추상화에 의존하는지 예제를 통해 살펴보자. 예를 들어 `k in M`과 같은 문법을 지원하는 `__contains__` 메소드는 다음과 같이 구현된다.

```python
def __conatins__(self, k):
    try:
        self[k]               # access via __getitem__ (ignore result)
        return True
    except KeyError:
        return False          # attempt failed
```

`setdefault` 메소드에서도 비슷한 방식을 이용한다.
```python
def setdefault(self, k, d):
    try:
        return self[k]             # if __getitem__ succeeds, return value
    except:                        # otherwise:
        self[k] = d                # set default value with __setitem__
        return d                   # and return that newly assigned value

`MutableMapping` 클래스의 다른 구상 메소드의 구현은 연습문제로 남긴다.

### 10.1.4 Our MapBase Class
우리는 이번 챕터와 다음 챕터를 통해서 맵 ADT의 서로 다른 여러가지 구현을 제공할 것이고, 그러면서 여러 자료 구조에서 나타나는 장점과 단점의 상충관계를 설명할 것이다. 아래의 그림은 그 클래스들의 미리보기라 할 수 있다.
<img width="600" alt="figure-10.2" src="https://user-images.githubusercontent.com/20944657/36953627-91608d8a-205f-11e8-9d58-95f672690dfd.png">

`MutableMapping` 추상 기초 클래스는 맵을 구현할 때 아주 유용하지만, 코드 재활용성을 증대시키기 위해서 우리는 `MutableMapping`의 서브클래스인 `MapBase` 클래스를 새로 정의할 것이다. 이 `MapBase` 클래스는 컴포지션 디자인 패턴을 지원하기 위해 고안되었다. 구체적으로 말하자면, `MapBase` 클래스는 `MutableMapping` 클래스의 유용한 메소드들을 상속받으면서도 추가적으로 nonpublic 중첩 클래스인 `_Item` 클래스를 정의한다. 이 `_Item` 클래스의 인스턴스는 key와 value를 동시에 저장할 수 있다. 이 클래스는 Section 9.2.1에서 봤던 `PriorityQueueBase` 클래스의 `_Item` 클래스와 유사하지만, 맵에서는 key을 이용한 비교를 지원한다는 점에서 차이가 있다. 맵을 구현할 때는 항상 동등성(equality)의 개념을 잘 정의해서 새로 항목을 추가할 때 이미 저장된 key 중에 새로 추가하려는 key와 같은 것이 있는 지 확인할 수 있게끔 해줘야 한다. 또한 `<` 연산자를 이용한 key 사이의 비교는 이후 **sorted map ADT**(Section 10.3)에서 중요하게 등장할 것이다.

In [4]:
from collections import MutableMapping
class MapBase(MutableMapping):
    """Our own abstract base class that includes a nonpublic _Item class."""
    
    #--------------------- nest _Item class -------------------------------
    class _Item:
        """Lighweight composite to store key-value pairs as map items."""
        __slots__ = '_key', '_value'
        
        def __init__(self, k, v):
            self._key = k
            self._value = v
            
        def __eq__(self, other):
            return self._key == other._key    # compare items based on their keys
        
        def __ne__(self, other):
            return not (self == other)        # opposite of __eq__
        
        def __lt__(self, other):
            return self._key < other._key     # compare items based on their keys

### 10.1.5 Simple Unsorted Map Implementation
`MapBase` 클래스를 이용해서 매우 간단한 맵 ADT의 구현을 해보자. 아래의 코드는 파이썬 리스트 내에 key-value 페어를 임의의 순서로 저장하는 `UnsortedTableMap` 클래스 예제이다. 

In [7]:
class UnsortedTableMap(MapBase):
    """Map implementation using an unordered list."""
    
    def __init__(self):
        """Create an empty map."""
        self._table = []
        
    def __getitem__(self, k):
        """Return value associated with key k (raise KeyError if not found)."""
        for item in self._table:
            if k == item._key:
                return item._value
        raise KeyError('Key Error: ' + repr(k))
    
    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present."""
        for item in self._table:
            if k == item._key:                                   # Found a match:
                item._value = v                                  # reassign value
                return                                           # and quit
        # did not find match for key
        self._table.append(self._Item(k,v))
        
    def __delitem__(self, k):
        """Remove item associated with key k (raise KeyError if not found."""
        for j in range(len(self._table)):
            if k == self._table[j]._key:
                self._table.pop(j)
                return
        raise KeyError('Key Error: ' + repr(k))
        
    def __len__(self):
        """Return number of items in the map."""
        return len(self._table)
    
    def __iter__(self):
        """Generate iteration of the map's keys."""
        for item in self._table:
            yield item._key

## 10.2 Hash Tables
이번 섹션에서는 파이썬 `dict` 클래스의 구현에 실제로 사용되고 있으면서, 맵을 구현하기 위한 가장 실용적인 자료 구조 중 하나인 **해쉬 테이블(table)**을 소개한다. 직관적으로, 맵 $M$은 `M[k]`와 같이 키를 인덱스로 이용한다. 예를 들어 맵 $M$이 $N \geq n$인 $N$에 대해 $0$부터 $N-1$까지의 정수 중에서 인덱스를 갖는 $n$개의 항목을 갖는다고 하자. 이 경우에 아래와 같은 **lookup table**을 이용해서 맵을 표현할 수 있다.

<img width="600" alt="figure-10.3" src="https://user-images.githubusercontent.com/20944657/36954280-37019dc0-2064-11e8-9516-10d7db25132f.png">

이러한 표현 방식을 이용할 경우 우리는 테이블의 $k$ 인덱스에 key $k$와 연결된 값을 저장한다(빈 슬롯을 표현하는 방법이 있다고 가정하자). 이 때 기본적인 `__getitem__`, `__setitem__`, `__delitem__`은 $O(1)$ worst-case 시간에 실행될 수 있다. 이러한 프레임워크를 더 일반적인 맵의 환경으로 확장하려면 두 가지 문제가 있다. 첫째, $N \gg n$일 경우 길이가 $N$인 배열을 유지하는 것은 비효율적이다. 둘째, 맵의 key가 정수일 것이라는 가정은 일반적이지 못하다.

해쉬 테이블이 이러한 문제를 해결하기 위해 사용한 새로운 개념은 일반적인 키를 테이블 인덱스로 매핑하기 위해 **해쉬 함수(hash function)**을 이용한 것이다. 이상적이라면 해쉬 함수에 의해서 key가 $0$부터 $N-1$까지 잘 분포되어야 할 것이지만, 실제로는 두개 이상의 key가 같은 인덱스로 매핑된다. 따라서 우리는 아래와 같이 테이블을 **bucket array**로 개념화한다. 각각의 버켓은 해쉬 함수로 인해 특정 인덱스로 오게 된 항목들의 집합을 관리한다(공간을 아끼기 위해 빈 버켓은 None으로 교체된다).

<img width="600" alt="figure-10.3" src="https://user-images.githubusercontent.com/20944657/36954440-651a8680-2065-11e8-90c8-8235e70cfc18.png">

### 10.2.1 Hash Functions
**해쉬 함수(hash function)**, $h$, 의 목표는 key $k$를 $[0, N-1]$ 안의 정수로 매핑하는 것이다. 이 때 $N$은 해쉬 테이블을 위한 bucket array의 수용능력(capacity)이다. 이제 맵은 해쉬 함수 값 $h(k)$를 이용해서 $k$ 대신 $h(k)$를 bucket array $A$의 인덱스로 이용한다. 즉, $(k,v)$를 버켓의 $A[h(k)]$에 저장한다.

만약 같은 해쉬값을 갖는 두 개 이상의 key가 존재할 경우 두 다른 항목이 $A$의 같은 버켓에 매핑된다. 이 경우 **충돌(collision)**이 일어났다고 한다. collision을 다루는 방법은 여러가지가 있고 나중에 이를 자세히 다룰 것이다. 그러나 지금 간단히 말하자면 가장 좋은 전략은 애초에 collision이 일어날 일이 없게 하는 것이다. 우리는 collision을 충분히 최소화하면서 맵의 key를 매핑해주는 해쉬 함수를 "좋은" 해쉬 함수라고 말한다. 또 실용적인 이유에서 볼 때에는 해쉬 함수가 계산하기에 빠르고 쉽기를 원하기도 한다.

일반적으로는 해쉬 함수의 평가 $h(k)$가 두 부분으로 이루어져있다고 본다. 첫번째는 key $k$를 정수로 매핑해주는 **hash code**이고, 두번째는 해쉬 코드를 인덱스 범위 $[0, N-1]$ 내로 매핑해주는 **압축 함수(compression function)**이다.

<img width="400" alt="figure-10.5" src="https://user-images.githubusercontent.com/20944657/36954635-b05f3cd4-2066-11e8-9c0a-4b3af575a116.png">

이렇게 해쉬 함수를 두 부분으로 나누는 접근방식의 장점은 해쉬 코드와 관련된 부분이 해쉬 테이블의 구체적인 크기와 무관하게 된다는 점이다. 이는 어떠한 크기의 해쉬 테이블에도 사용될 수 있는 일반적인 해쉬 코드의 발전을 가능하게 한다. 압축 함수만이 해쉬 테이블의 크기에 의존한다. 이렇게 되면 현재 맵에 저장된 항목들의 수에 따라 해쉬 테이블 내의 버켓 배열이 동적으로 리사이징되는 것이 가능해져서 굉장히 편리하다(See Section 10.2.3.)

### Hash Codes
해쉬 함수가 처음으로 취하는 행동은 맵의 임의의 key $k$를 이용해서 $k$의 **해쉬 코드**라 불리는 정수를 계산하는 것이다. 이 정수는 $[0, N-1]$ 안에 있어야 할 필요가 없고, 음수일 수도 있다. 우리는 키에 할당된 해쉬 코드의 집합이 가능한 한 충돌(collision)을 피하기를 원한다. 만약 해쉬 코드에서 충돌이 일어난다면 압축 함수가 충돌을 피할 수 있는 가능성은 없다. 이제 해쉬 코드의 이론을 다뤄보고 실제 해쉬 코드가 파이썬에서 어떻게 구현되는지 알아보자.

### Treating the Bit Representation as an Integer
우리의 해쉬 코드 정수의 비트 이하의 비트를 이용해서 표현되는 자료형 $X$는 그 비트의 정수 표현을 해쉬 코드로 이용하는 것이 가능하다. 예를 들어 key 314의 해쉬 코드는 314가 될 수 있다. 3.14와 같은 부동소수점의 해쉬 코드는 부동소수 표현의 비트를 정수로 표현하고 그 정수를 해쉬 코드로 쓸 수 있다.

만약 그 비트 표현이 해쉬 코드의 비트 수보다 길다면 위와 같은 방식은 사용할 수 없다. 예를 들어 파이썬은 32비트 해쉬 코드를 이용한다. 만약 부동소수가 64비트로 표현된다면 그 비트는 해쉬 코드로 이용할 수 없다. 이를 피하는 방법 중 하나는 high-order 32비트만 이용하는 것이다. 물론 이렇게 하면 원래 key의 50%가 무시될 뿐 아니라 많은 key가 일부 비트만 빼고 동일하다면 collision을 피할 수도 없게 될 것이다.

더 나은 접근법은 64비트 키의 high-order와 low-order 부분을 어떻게든 잘 조합해서 32비트 해쉬 코드를 만드는 것이다. 간단한 방법 중 하나는 각각이 32비트 수이므로 둘을 더하는 것이다(오버플로우는 무시한다). 아니면, 두 비트의 `exclusive-or`을 취하는 방법도 있다. 이렇게 두 요소를 합하는 접근방식은 그 이진 표현이 32비트 정수의 $n$-튜플, $(x_{0}, x_{1}, ... , x_{n-1})$,로 표현될 수 있는 임의의 객체 $x$에 대해서도 확장될 수 있다. 예를 들어 $x$의 해쉬 코드는 $\sum_{i=0}^{n-1} x_{i}$나 $x_{0} \oplus x_{1} \oplus \cdots \oplus x_{n-1}$로 나타낼 수 있다. 이 때 $\oplus$는 파이썬에서 `^`로 표현되는 `exclusive-or` 연산을 의미한다.

### Polynomial Hash Codes
위의 합이나 `exclusive-or` 해쉬 코드는 튜플 $(x_{0}, x_{1}, ..., x_{n-1})$로 표현될 수 있는 객체이지만 그 튜플 내에서 $x_{i}$의 순서가 중요한 경우에는 좋은 선택이 아니다. 예를 들어 문자열 $s$안의 유니코드 값을 합하는 16비트 해쉬 코드를 생각해보자. 이 해쉬 코드는 불행히도 많은 collision을 발생시킨다. 예를 들어 "temp01"과 "temp10"은 이 함수에 의해 똑같은 해쉬 코드를 갖게 된다. "stop", "tops", "pots", "spot"도 마찬가지이다. 이러한 문제를 해결하기 위한 좋은 방법으로는 $x_{i}$의 순서를 고려하는 해쉬 코드를 이용하는 것이 있다. 그러한 방식을 이용하는 더 나은 방법으로는 0이 아닌 상수 $a \neq 1$을 고른 후,

$x_{0}a^{n-1} + x_{1}a^{n-2} + \cdots + x_{n-2}a + x_{n-1}$ 을 해쉬 코드로 이용하는 것이 있다.

수학적으로 말하면 이는 단순히 $(x_{0}, x_{1}, ..., x_{n-1}$을 계수로 이용하는 $a$의 다항식일 뿐이다. 따라서 이 해쉬코드는 **polynomial hash code**라고 불린다. Horner's rule을 이용하면 이 다항식은 다음과 같이 계산할 수 있다.

$x_{n-1} + a(x_{n-2} + a(x_{n-3} + \cdots + a(x_{2} + a(x_{1} + ax_{0}))\cdots))$

직관적으로, 이 다항 해쉬 코드는 각각의 요소의 영향력을 퍼트리기 위한 방법으로 제곱(power)을 이용한다. 물론 일반적인 컴퓨터에서 다항식의 계산이 해쉬 코드의 비트 수를 이용해서 이루어지므로 오버플로우가 발생할 수 있다. 그러나 우리는 다른 key와 $x$의 차이를 구분하는 것에 관심이 있으므로 그러한 오버플로우는 무시해도 된다. 그럼에도 항상 주의해야 할 것은 오버플로우가 일어나고 있다는 사실을 인지하고, 만약 오버플로우가 일어날 경우 low-order nonzero bit를 이용할 수 있게끔 상수 $a$를 잘 정해줘야 한다는 것이다.

예를 들어 실험 연구에 따르면 영어 단어 문자열을 갖고 작업할 때에는 33, 37, 39, 41이 $a$의 값으로 적합하다고 알려져 있다. 50,000개 이상의 영어 단어 리스트를 갖고 작업할 때 33, 37, 39, 41의 $a$ 값을 이용하면 7개 이하의 collision이 발생한다.

### Cyclic-Shift Hash Codes
다항 해쉬 코드의 변종으로는 $a$를 곱하는 대신 부분합에 특정 비트 수 만큼의 cyclic shift를 취하는 것이 있다. 예를 들어, 32비트 값 **00111**101100101101010100010101000에 5 비트 cyclic shift를 하면 왼쪽 5개 비트를 오른쪽으로 옮겨서 다음의 32비트 값 101100101101010100010101000**00111**이 나오게 된다. 이러한 연산은 산술적으로 볼 때 큰 의미가 없지만 비트를 다양하게 만든다는 목표는 충실히 이행한다. 파이썬에서 이러한 연산을 하기 위해서는 비트 연산 `<<`와 `>>`를 신중하게 이용하면 된다.

In [62]:
def hash_code(s):
    mask = (1 << 32) - 1                   # limit to 32-bit integers
    h = 0
    for character in s:
        h = (h << 5 & mask) | (h >> 27)    # 5-bit cyclic shift of running sum
        h += ord(character)                # add in value of next character
    return h

전통적인 다항 해쉬 코드와 마찬가지로 cyclic-shift 해쉬 코드를 이용할 때에도 fine-tuning이 필요하다. 특히 쉬프트 연산의 크기를 결정해야 한다. 위에서 쉬프트의 크기를 5 비트로 한 것은 230000개가 넘는 영단어에 대한 실험을 통해 도출된 결과이다.

<img width="300" alt="table-10.1" src="https://user-images.githubusercontent.com/20944657/36956084-aacde78a-206f-11e8-8d1d-81bd1afbc1c6.png">


### Hash Codes in Python
파이썬에서 해쉬 코드를 계산하는 표준적인 메커니즘은 built-in 함수인 `hash(x)`를 이용하는 것이다. `hash(x)`는 객체 $x$에 대한 해쉬 코드를 반환한다. 그런데 파이썬에서는 오직 **immutable**한 자료형만 hashable하다고 취급한다. 이러한 제약은 객체가 존재하는 내내 해쉬 코드가 상수로 유지되게끔 하기 위한 것이다. 이는 객체를 해쉬 테이블의 key로 이용하기 위한 매우 중요한 성질이다. 만약 key가 해쉬 테이블에 추가되었는데 추가 됐을 때와 다른 해쉬 코드를 가진 key로 탐색을 한다고 하면, 잘못된 버켓을 찾게 될 것이다.

파이썬의 built-in 자료 형 중에서도 immutable `int`, `float`, `str`, `tuple`, `frozenset` 클래스들은 `hash` 함수를 통해 robust한 해쉬 코드를 만들어낼 수 있다. `string`의 해쉬 코드는 `exclusive-or` 연산 대신 덧셈을 이용한다는 점을 빼면 앞서 다룬 다항 해쉬 코드와 비슷한 테크닉을 이용해서 만들 수 있다. 만약 위에서 제시했던 표와 비슷한 실험을 하면 230000개의 영단어에 대해 8개의 문자열만 서로와 충돌하게 된다. `tuple`의 해쉬코드는 튜플의 개별 원소의 해쉬 코드의 조합을 이용하는 비슷한 테크닉으로 계산할 수 있다. `frozenset`을 해쉬할 때는 원소에 순서가 없으므로 `shift` 없이 `exclusive-or`을 하는 것이 자연스럽다. `list`와 같이 mutable한 타입 `x`에 `hash(x)`를 호출하면 `TypeError`가 발생한다.